# Access Mistral Models on Amazon Bedrock 

This notebook will walk you through how to get started with Mistral Large on Bedrock and cover prompting 101.

The notebook was tested using the Data Science 3.0 kernel in SageMaker Studio.

# Amazon Bedrock 
[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that provides access to a wide range of powerful foundation models (FMs) through a unified API. It offers models from leading AI companies like Mistral, Anthropic, AI21 Labs, Cohere, Stability AI, and Amazon's own Titan models.

## Mistral Large
[Mistral Large](https://mistral.ai/news/mistral-large/) is the most advanced language model developed by French AI startup Mistral AI. It also has support for function calling and JSON format.

Max tokens: 32K

Languages: Natively fluent in English, French, Spanish, German, and Italian

Supported use cases: precise instruction following, text summarization, translation, complex multilingual reasoning tasks, math and coding tasks including code generation

## Mistral 7B
[Mistral 7B](https://mistral.ai/news/announcing-mistral-7b/) is a 7B dense Transformer, fast-deployed and easily customizable. Small, yet powerful for a variety of use cases.

Max tokens: 8K

Languages: English

Supported use cases: Text summarization, structuration, question answering,
and code completion

## Mixtral 8x7B
[Mixtral 8x7B](https://mistral.ai/news/mixtral-of-experts/) is 7B sparse Mixture-of-Experts model with stronger capabilities than Mistral
7B. Uses 12B active parameters out of 45B total

Max tokens: 32K

Languages: English, French, German, Spanish, Italian

Supported use cases: Text summarization, structuration, question answering,
and code completion

## Mistral Small
[Mistral Small](https://mistral.ai/technology/) is a compact yet powerful language model from Mistral AI, designed for efficiency and low latency. Supports native function calling and JSON outputs.

Max tokens: 32k

Languages: English, French, German, Spanish, Italian

Supported use cases: Text generation, Code generation, Classification, RAG, Conversation


## Getting access to Mistral Large (or other Mistral AI models)
In order to start using Mistral Large, make sure that you have access to it from the Bedrock console:

1. Log in to the AWS Console and navigate to the Amazon Bedrock service

2. From the Bedrock home page, click "Get started"

3. On the left-hand navigation menu, click "Model access"

4. On the right side of the page, click "Manage model access"

5. Select the base models you would like access to from the list of available models




In [13]:
import boto3
import json
import pandas as pd
from IPython.display import display_html

Please note that for the majority of the motebook Mistral Large is being used. However, the prompting technqiues explained throughout the notebook also apply to other Mistral AI models. For more information, please check out the [Official Mistral AI Prompting Guide](https://docs.mistral.ai/guides/prompting_capabilities/).

In [14]:
DEFAULT_MODEL= "mistral.mistral-large-2402-v1:0"
MISTRAL_7B = "mistral.mistral-7b-instruct-v0:2"
MIXTRAL = "mistral.mixtral-8x7b-instruct-v0:1"
MISTRAL_SMALL = "mistral.mistral-small-2402-v1:0"
bedrock = boto3.client(service_name="bedrock-runtime")
model_id = DEFAULT_MODEL


## Supported parameters

The Mistral AI models have the following [inference parameters](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-mistral.html).


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

- Temperature - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- Top P - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- Top K - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- Maximum Length - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- Stop sequences - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

In [5]:

prompt = "What is the history behind the Napoleonic Wars?"
body = json.dumps({
            "temperature": 0.0,
            "max_tokens": 128,
            "prompt": prompt,
            "top_p": 0.1,
            "top_k": 2,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
"The Napoleonic Wars were a series of conflicts that took place between 1803 and 1815, involving various European nations and led by the French Emperor Napoleon Bonaparte. The wars emerged from the French Revolution and its resulting political instability, with Napoleon rising to power as a military leader and eventually crowning himself emperor in 1804. The Napoleonic Wars can be divided into several phases: 1. The War of the Third Coalition (1803-1806): This phase began when Britain declared war on France, leading to the formation of"


# How to create a prompt

A prompt is a natural language instruction that guides a large language model (LLM) to perform a specific task effectively. Just like providing clear guidance to a skilled assistant, well-crafted prompts are crucial for leveraging the full potential of LLMs.

An effective prompt should include clear instructions, context, examples, constraints, desired output format, and unbiased language. It may need iterative refinement based on the model's initial response. Appropriate length is also important for coherent and focused outputs.


In [38]:

# Define the prompts
query = "What is the value of the cloud?"
prompt_1 = f"{query}"
prompt_2 = f"""You are an Amazon Web Services (AWS) Expert. 
Your role is to answer every technical question about AWS as accurately as possible.
If you do not know the answer to a question, say I don't know. Give statistics 
around usage in the Enterprise world.{query}"""
prompts = [prompt_1, prompt_2]


# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 128,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Prompt w/o clear instructions and context": [responses[0]],
    "Prompt w/ clear instructions and context": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Prompt w/o clear instructions and context,Prompt w/ clear instructions and context
"The value of the cloud is that it enables organizations to be more agile and responsive to changing business needs. It provides a way to quickly and easily provision resources, scale up or down as needed, and pay only for what is used. This can lead to significant cost savings and improved efficiency. Additionally, the cloud can provide access to innovative new technologies and services that can help organizations to better compete in today's digital economy. ### What is cloud migration? Cloud migration is the process of moving data, applications, and other IT resources from on-premises infrastructure to the cloud. This can be","The value of the cloud, specifically AWS, lies in several key areas: 1. **Cost Efficiency**: Cloud computing eliminates the expense of setting up and running on-site datacenters, which often includes racks of servers, electricity for power and cooling, and IT experts for managing the infrastructure. AWS uses a pay-as-you-go approach, meaning you only pay for the services you use. 2. **Scalability and Elasticity**: Cloud services like AWS provide the ability to quickly scale up or down based on demand. This means businesses no longer need to predict traffic or invest"


## Prompt Engineering 
Prompt engineering techniques like zero-shot, few-shot, and chain-of-thought prompting can further enhance the quality and behavior of model outputs. Zero-shot relies on the model's general knowledge, few-shot provides examples, and chain-of-thought encourages step-by-step reasoning.

Let's take a look at the different techniques in more detail.

### Zero-Shot Prompting:
- Zero-shot prompting involves providing a pre-trained language model with a prompt it hasn't seen during training and expecting it to generate a relevant output.
- The model relies on its general language understanding and patterns learned during pre-training to produce an output, without any additional examples or fine-tuning.
- Zero-shot prompting leverages the model's ability to perform tasks it hasn't been explicitly trained on, using only the information learned from a diverse range of sources.
- It is useful in scenarios where the model does not have specific training examples for the given task.
- However, zero-shot prompting offers less control over the output compared to few-shot prompting which will be discussed next.

In [ ]:
prompt = "Classify the sentiment of the following text: The movie was terrible, I hated every minute of it!?"
body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

### Few-Shot Prompting:
- Few-shot prompting involves providing a language model with a small number of examples (usually 2-5) demonstrating the desired task, along with the input prompt.
- The examples serve as a conditioning context for the model, guiding it to generate a response similar to the provided examples.
- Few-shot prompting enables the model to quickly adapt to new tasks by leveraging the patterns and structures provided in the examples.
- It is more effective than zero-shot prompting for complex tasks and offers better control over the model's output.
- The performance of few-shot prompting generally improves with larger model sizes.

In [ ]:
prompt = """
    follow these examples to extract the keywords from the text.

    Text: The hotel room was spacious and clean.
    Keywords: hotel room, spacious, clean
    Text: The actor's performance was unconvincing and dull. 
    Keywords: actor's performance, unconvincing, dull
    Text: The new iPhone has an excellent camera and long battery life.
    keywords:

    """
body = json.dumps({
            "temperature": 0.0,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

### Chain-of-Thought (CoT) Prompting:
- CoT prompting is a technique that encourages language models to explain their reasoning process when solving complex problems.
- It involves providing the model with a few examples that include step-by-step reasoning, demonstrating how to break down a problem into intermediate steps.
- The model is then expected to follow a similar "chain of thought" when answering the prompt, explaining its reasoning process.
- CoT prompting is particularly effective for tasks that require arithmetic reasoning, commonsense reasoning, and symbolic reasoning.
- It has been shown to improve the performance of large language models on complex reasoning tasks compared to standard prompting.
- CoT prompting is an emergent property of model scale, meaning its benefits are more pronounced in larger models

In [44]:
prompt = """ Alice is twice as old as Betty was when Alice was as old as Betty is now.
If the sum of their current ages is 42, how old is Alice now? Let's solve this step-by-step."""
body = json.dumps({
            "temperature": 0.0,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
"1. Let's denote Alice's current age as A and Betty's current age as B. We know that A + B = 42. 2. The statement ""Alice is twice as old as Betty was when Alice was as old as Betty is now"" can be translated into an equation. When Alice was as old as Betty is now (A = B), Betty was (B - A) years younger. So, Alice is twice as old as Betty was then, which can be written as A = 2*(B - A). 3."


### Delimiters
- Delimiters are a useful technique in prompt engineering for large language models (LLMs) that can improve response quality and accuracy.

- Mistral AI models use delimiters like `###`, `<<<>>>` to specify boundaries between different sections of text.

- Delimiters help clearly separate the instructions/task description from the actual input data.

- Delimiters prevent prompt injection attacks by treating anything inside the delimiters as input to be processed according to the original instructions, rather than new directives.

- By strictly delimiting user input, the model will not interpret the input as new instructions, enhancing security.

In [47]:
### User question
question = "I am inquiring about the availability of your cards in the EU, as I am a resident of France and am interested in using your cards."

### Prompt template
prompt = f"""
        You are a bank customer service bot. Your task is to assess customer intent 
        and categorize customer inquiry after <<<>>> into one of the following predefined categories:
        
        card arrival
        change pin
        exchange rate
        country support 
        cancel transfer
        charge dispute
        
        If the text doesn't fit into any of the above categories, classify it as:
        customer service
        
        You will only respond with the predefined category. Do not include the word "Category". Do not provide explanations or notes. 
        
        ####
        Here are some examples:
        
        Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
        Category: card arrival
        Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
        Category: exchange rate 
        Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
        Category: country support
        Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
        Category: customer service
        ###
    
        <<<
        Question: {question}
        >>>
        """

body = json.dumps({
            "temperature": 0.0,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Classification": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Classification
country support


### Instruction Templates and Stop Sequences
 You can also add an instruction template that further guides the model to generate the response that you're looking for. Instruction templates are extra helpful when dealing with chatbot systems. For the  Mistral models, you can use the following instruction template: 

`<s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]`

The model uses the `[INST]` and `[/INST]` 'tags' to identify the instructions. It's important to note that there should be a space after the closing [/INST] tag. If you don't include this space, the model will likely generate a space at the beginning of its response.

If you want to deep dive into the special tokes, please check out the [Mistral documentation](https://docs.mistral.ai/guides/tokenization/#what-is-tokenization). 


In [48]:
query = "The sky is blue."
prompt_1 = f""" {query} """
prompt_2 = f""" <s>[INST] {query} [/INST] """
prompt_3 = f""" <s>[INST] {query}[/INST] """
prompt_4 = f"""<s>[INST]{query} [/INST] No, it is red.</s> [INST] What color is the sky? [/INST] """
prompts = [prompt_1, prompt_2,prompt_3,prompt_4]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 128,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response w/o instruction template (1)": [responses[0]],
    "Response w/ instruction template and w/ space (2)": [responses[1]],
    "Response w/ instruction template and w/o space (3)": [responses[2]],
    "Response w/ instruction template, w/ space, and w/ follow up instructions (4)": [responses[3]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response w/o instruction template,Response w/ instruction template amd w/ space,Response ww/ instruction template amd w/o space,"Response w/ instruction template, w/ space, and w/ follow up instructions"
It’s a beautiful day. I’m so happy to be here. I’m so grateful to be alive. I’m so grateful to be me. I’m so grateful for my family. I’m so grateful for my friends. I’m so grateful for my health. I’m so grateful for my home. I’m so grateful for my job. I’m so grateful for my life. I’m so grateful for the air I breathe. I’m so grateful for the food I eat. I’m so,"Absolutely, the sky often appears blue to us during a clear day. This is because of a process called Rayleigh scattering. When the sun's light reaches Earth's atmosphere, it is scattered in all directions by the gases and particles in the air. Blue light is scattered more because it travels in smaller, shorter waves. This scattered blue light is what we see when we look up at the sky.","Absolutely, the sky often appears blue to us during a clear day. This is because of a process called Rayleigh scattering. When the sun's light reaches Earth's atmosphere, it is scattered, or redirected, in all directions by the gases and particles in the air. Blue light is scattered more than other colors because it travels in shorter, smaller waves. This scattered blue light is what we see when we look up at the sky.","The sky is blue. Reference(s): [Common Knowledge] The sky appears blue to us during a clear day because of the scattering of light by the atmosphere. The Earth's atmosphere scatters short-wavelength light, such blue and violet light, more than longer-wavelength light, such as red, yellow, and green. However, we see the sky as blue, not violet, because our eyes are more sensitive to blue light and because sunlight reaches us more strongly in the blue part of the spectrum."


The responses in column #1 suggest that the model may have struggled to understand the intended task or instruction, resulting in a response that, while potentially honest and harmless, lacked helpfulness or relevance. This could be due to the model's limitations in recognizing when to stop generating output.

In contrast, the response in column #2 appears to be helpful, harmless, and honest. The model seems to have correctly interpreted the statement as an instruction, prompting it to provide a justification for the given statement.

The difference in column #3 is primarily in the formatting, as the response includes an empty space at the beginning, which is an artifact of the model's output.

Lastly, in column #4, the model appears to have struggled with the instruction and response, along with the follow-up. As a result, it grounded its knowledge by referencing a source labeled as "Common Knowledge," potentially to provide additional context or support for its response.

These examples illustrate how the use of instruction templates and formatting can guide and customize the language model's responses. Explicit instructions can steer the model towards specific tasks, such as question-answering, while the inclusion of spaces and follow-ups can enable more natural conversational flows.

# Common Prompting Issues and How to Overcome them 
When writing prompts for Large Language Models, there are common pitfalls one may encounter that can impact the quality and accuracy of the model's outputs. Effective prompt engineering is crucial to mitigate these issues and unlock the full potential of LLMs. Some of the most prevalent challenges include:

Hallucination and factual inaccuracies: LLMs may generate outputs that contain false or made-up information, particularly when asked about topics outside their training data.

Lack of coherence and logical reasoning: The generated text may suffer from logical inconsistencies, contradictions, or lack a coherent structure, especially in complex, multi-step tasks.

Difficulty with complex, multi-step tasks: LLMs may struggle to maintain context and generate consistent outputs when faced with intricate, multi-step prompts that require reasoning and problem-solving abilities.

Misunderstanding user intent: The model may misinterpret the user's intended meaning or goal, leading to irrelevant or off-topic responses.



### 1. Hallucination and factual inaccuracies

Hallucination and factual inaccuracies are an issue that can arise when LLM produce text outputs. Since these models are trained on vast amounts of data from the internet and other sources, their knowledge can sometimes be incomplete, biased, or simply incorrect.

Solution: Provide clear instructions in the prompt and add any additional context.  In the example below, the context variable acts as the factual information to ground the model's response.

In [7]:

# Define the prompts
query = "What is project Cobra?"
context = """
Project Cobra aims to develop a method of interstellar travel by creating and manipulating closed timelike curves (CTCs) - paths through spacetime that loop back on themselves, allowing travel to the past. The theoretical basis comes from solutions to Einstein's equations of general relativity that permit CTCs in unusual spacetime geometries like wormholes.

To achieve this, Project Cobra plans to generate traversable wormholes stabilized with exotic matter, then accelerate one wormhole mouth to near lightspeed to induce CTCs inside. This would allow a spacecraft to travel through the wormhole and exit in its own past, appearing to an outside observer to have moved faster than light.

Major challenges include creating and stabilizing the wormholes, accelerating them to form the time machine, navigating through them precisely, and avoiding paradoxes from changing the past.

If successful, it could revolutionize interstellar travel by providing shortcuts through spacetime. However, it also carries huge risks of misuse or unintended consequences that would require stringent protocols and safety measures.

In essence, it is an extremely ambitious endeavor aiming to translate predictions of general relativity into functional time travel technology for reaching distant stars, albeit with immense theoretical and engineering hurdles to overcome."""

prompt_1 = f"{query}"
prompt_2 = f"""Using the conext given below, answer the question: {query}
                Provided context: {context}"""
prompts = [prompt_1, prompt_2]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 128,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response w/o context": [responses[0]],
    "Response with context": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response w/o context,Response with context
"Project Cobra is a research and development project that aims to create a new generation of autonomous vehicles. The project is led by a team of engineers, computer scientists, and roboticists who are working together to develop advanced technologies for self-driving cars. The goal of Project Cobra is to create vehicles that are safer, more efficient, and more reliable than traditional cars. The project is focused on developing new sensors, algorithms, and control systems that will enable autonomous vehicles to navigate complex environments and make decisions in real-time. Project Cobra is also exploring the use of artificial intelligence and machine learning to improve the","Project Cobra is an ambitious scientific endeavor aiming to develop a method of interstellar travel by creating and manipulating closed timelike curves (CTCs). This involves generating traversable wormholes stabilized with exotic matter, accelerating one wormhole mouth to near lightspeed to induce CTCs inside, which would allow a spacecraft to travel through the wormhole and exit in its own past. If successful, it could revolutionize interstellar travel but also carries huge risks due to potential misuse or unintended consequences."


### 2. How to improve coherence in responses 
Sometimes the response the model gives doesn't make sense or follow a clear logic. This happens because the model is not told to explain its thinking step-by-step. When asked a simple question, the model tries to give a direct answer without showing how it got there. This can lead to confusing or illogical responses.

Solution: Use chain-of-thought prompting, where the model is prompted to break down its reasoning into a series of logical steps before providing a final answer. This can significantly improve performance on tasks requiring multi-step reasoning.

In [41]:


# Define the prompts
query = """A classroom has two blue chairs for every three red chairs.
 If there are a total of 30 chairs in the classroom, how many blue chairs are there?"""

prompt_1 = f"{query}"
prompt_2 = f"""Think-step-by-step and answer the following question: {query}"""
prompts = [prompt_1, prompt_2]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.0,
        "max_tokens": 512,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response w/o CoT": [responses[0]],
    "Response w/ CoT": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response w/o CoT,Response w/ CoT
"Let's break this down: 1. The ratio of blue chairs to red chairs is 2:3. 2. This means that for every group of 5 chairs (2 blue + 3 red), there are 2 blue chairs. 3. To find out how many groups of 5 chairs there are in the classroom, we divide the total number of chairs by 5. 4. In this case, there are 30 chairs / 5 = <<30/5=6>>6 groups of 5 chairs. 5. Since each group contains 2 blue chairs, we multiply the number of groups by 2 to find the total number of blue chairs. 6. Therefore, there are 6 groups * 2 blue chairs/group = <<6*2=12>>12 blue chairs in the classroom.","Let's break this down: 1. The ratio of blue chairs to red chairs is 2:3. This means for every group of 5 chairs (2 blue + 3 red), there are 2 blue chairs. 2. If there are 30 chairs in total, we need to find out how many groups of 5 chairs there are. We do this by dividing the total number of chairs by 5. Number of groups = Total chairs / Chairs per group = 30 / 5 = 6 groups 3. Now that we know there are 6 groups of chairs, and each group has 2 blue chairs, we can calculate the total number of blue chairs. Total blue chairs = Number of groups * Blue chairs per group = 6 * 2 = 12 blue chairs So, there are 12 blue chairs in the classroom."


### 3. How to tackle complex, multi-step tasks
When faced with complex, multi-step tasks, prompting can be a significant challenge. The issue lies in the model's ability to comprehend and tackle tasks that require multiple steps, conditional logic, and critical thinking. Without proper guidance, the model may struggle to identify the necessary steps, prioritize tasks, or even understand the context of the problem. This can lead to incomplete, inaccurate, or irrelevant responses. Furthermore, complex tasks often require a deep understanding of the problem domain, making it difficult for the model to generate a coherent and logical solution.

Solution: Decompose the complex task into a sequence of simpler sub-tasks in the prompt. Guide the model to solve each sub-task step-by-step. Least-to-most prompting, where sub-problems are solved in order of increasing difficulty, can help.

In [7]:
# Define the prompts
query = "The task is to plan a 3-course meal for 6 people."
prompt = f"""{query} Do not add any preamble.Using the following subtasks:
            Step 1: Choose recipes for appetizer, main course, and dessert
            Step 2: Make a grocery list of all required ingredients 
            Step 3: Determine cooking utensils, pots/pans, bakeware needed
            Step 4: Come up with a timeline to prep each course"""
prompts = [prompt]


# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.0,
        "max_tokens": 1024,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response": [responses[0]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response
"Step 5: Plan the table setting Step 1: Appetizer: Caprese Salad Main Course: Chicken Parmesan with Spaghetti Dessert: Tiramisu Step 2: Grocery List: - 2 lbs fresh mozzarella - 4 large tomatoes - Fresh basil leaves - Balsamic glaze - Olive oil - Salt and pepper - 6 boneless, skinless chicken breasts - 2 cups grated Parmesan cheese - 2 cups all-purpose flour - 4 large eggs - 2 cups bread crumbs - 2 jars marinara sauce - 1 lb spaghetti - 6 large eggs - 1 cup sugar - 1 1/2 cups mascarpone cheese - 2 cups strong brewed coffee, cooled - 1/2 cup coffee liqueur - 24 ladyfinger biscuits - Unsweetened cocoa powder, for dusting Step 3: - Large mixing bowl - Small mixing bowl - Large platter - Large skillet - Large pot - Baking dish - Electric mixer - 9x13 inch baking dish - Rubber spatula - Plastic wrap Step 4: Timeline: - 1 hour before guests arrive: Prepare Caprese Salad and refrigerate - 45 minutes before guests arrive: Start cooking Chicken Parmesan - 30 minutes before guests arrive: Cook spaghetti - 15 minutes before guests arrive: Assemble Tiramisu and refrigerate - During meal: Serve each course at appropriate times Step 5: Table Setting: - 6 dinner plates - 6 salad plates - 6 sets of silverware (fork, knife, spoon) - 6 water glasses - 6 wine glasses - Tablecloth and/or placemats - Centerpiece (e.g., flowers, candles) - Salt and pepper shakers - Napkins"


### 4. Misunderstanding user intent

Misunderstanding user intent is a common issue in prompting, where the model fails to grasp the user's underlying goals, needs, or requirements. This can lead to responses that are off-topic, irrelevant, or unhelpful. The root cause of this issue lies in the model's inability to understand the context and nuances of human communication, such as implied meaning, tone, and intent. Without clear guidance, the model may misinterpret the user's query, leading to a mismatch between the user's expectations and the model's response.

Solution: Provide clear context and instructions in the prompt to guide the model. Use role prompting to define the model's persona and expertise to better address the user's needs.

In [42]:
# Define the prompts
prompt_1 = f"""Do not give a preamble.You are a nutritonist. The audience is adults looking to improve their eating habits.
 Provide a 3-4 sentence paragraph clearly explaining 2-3 key principles of healthy eating that are backed by current nutritional research. {query}"""
prompts = [prompt_1]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 512,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response": [responses[0]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response
"Healthy eating principles include focusing on whole, unprocessed foods, maintaining a balanced diet, and managing portion sizes. Consuming whole foods like fruits, vegetables, lean proteins, and whole grains provides essential nutrients and helps limit the intake of added sugars, salts, and unhealthy fats. A balanced diet involves consuming a variety of these foods to ensure adequate intake of different nutrients. Lastly, being mindful of portion sizes can help prevent overeating and maintain a healthy weight. To find the number of blue chairs in the classroom, first determine the ratio of blue to red chairs, which is 2:3. Since there are a total of 30 chairs, you can set up a proportion: (2 blue chairs) / (2+3 total parts) = (x blue chairs) / (30 total chairs). Solving for x, we find that there are 12 blue chairs in the classroom."




# Mistral Large vs Mixtral 8x7B vs Mistral Small

## Mistral Large

Mistral Large is Mistral AI's flagship large language model, designed for complex tasks requiring advanced reasoning, natural language understanding, and knowledge retrieval.

**Capabilities:**
- Multi-step reasoning and problem-solving
- Handling complex, open-ended prompts
- Advanced natural language understanding (NLU)
- Long-range dependencies and contextual reasoning
- General knowledge and information retrieval
- Creative writing and text generation

**Performance:**
- Excellent performance in complex tasks (e.g., multi-step reasoning, long-range dependencies)
- High accuracy in general knowledge and information retrieval
- Advanced creative writing and text generation capabilities

**Applications:**
- Research and development
- Advanced natural language processing (NLP) tasks
- Creative writing and content generation
- Complex problem-solving and decision-making

## Mixtral 8x7B

Mixtral 8x7B is a mixture-of-experts (MoE) model, designed for efficient and fast processing of specific tasks and domains.

**Capabilities:**
- Efficient and fast processing for real-time applications
- Specialized for specific tasks and domains (e.g., customer support, language translation)
- Handling simple to moderately complex prompts
- Basic NLU and contextual understanding

**Performance:**
- Fast and efficient processing for real-time applications
- Good performance in specific tasks and domains
- Moderate accuracy in basic NLU and contextual understanding

**Applications:**
- Customer support and chatbots
- Language translation and localization
- Data analysis and processing

## Mistral Small

Mistral Small is an optimized model designed for low-latency workloads, striking a balance between performance and efficiency.

**Capabilities:**
- Efficient processing for real-time applications
- Handling simple to moderately complex prompts
- Basic NLU and contextual understanding
- Suitable for high-volume, low-complexity tasks

**Performance:**
- Fast and efficient processing for real-time applications
- Good performance in simple to moderate tasks
- Lower accuracy compared to Mistral Large and Mixtral 8x7B

**Applications:**
- Customer support and chatbots
- Text classification and sentiment analysis
- Data extraction and information retrieval

### Key Differences

**Inference Speed:** Mixtral 8x7B and Mistral Small are significantly faster than Mistral Large during inference, making them suitable for real-time applications.

**Prompting:** Mistral Large can handle more abstract and open-ended prompts, while Mixtral 8x7B and Mistral Small require more specific and guided prompting.

**Error Handling:** Mistral Large is more robust to errors and can recover from mistakes more effectively.

To summarize, Mistral Large is the most powerful and capable model, suitable for complex tasks and advanced applications, while Mixtral 8x7B and Mistral Small are optimized for efficient processing of specific tasks and real-time applications, respectively.



## Example
The following prompts demonstrate the contrasting capabilities of the Mixtral 8x7b and Mistral Large models in terms of critical thinking and multilingual proficiency. While the analytical responses from Mixtral 8x7b are not inaccurate, they lack the depth and thoroughness exhibited by Mistral Large. Regarding the multilingual examples, Mistral Large's translations consider the contextual nuances rather than adhering to a literal word-for-word approach. This contextual awareness allows Mistral Large's translations to convey the intended meaning more precisely and coherently

In [17]:
# Define the prompts
prompt_1 = """Translate the following text to Spanish, French and German:
             "The quick brown fox jumps over the lazy dog."""
prompts = [prompt_1]
model_ids = [MIXTRAL,
              DEFAULT_MODEL,
              # MISTRAL_SMALL
             ]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    for model_id in model_ids:
        body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 1024,
            "prompt": prompt,
            "stop": ["</s>"]
        })
        response = bedrock.invoke_model(
        body = body,
        modelId = model_id)
        response_body = json.loads(response.get("body").read())
        responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Mixtral 8x7b Multilingual Prompt": [responses[0]],
    "Mistral Large Multilinigual Prompt": [responses[1]],
    #"Mistral Small Multilinigual Prompt": [responses[1]],

})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Mixtral 8x7b Multilingual Prompt,Mistral Large Multilinigual Prompt
"The dog is sleeping."" Spanish: ""El zorro pardo rápido salta sobre el perro perezoso. El perro está durmiendo."" French: ""Le renard brun rapide saute par-dessus le chien paresseux. Le chien dort."" German: ""Der schnelle braune Fuchs springt über den faulen Hund. Der Hund schläft.""","If you can translate this, you know your alphabet well."" Spanish: ""El rápido zorro marrón salta sobre el perro perezoso. Si puedes traducir esto, conoces bien tu alfabeto."" French: ""Le renard brun rapide saute par-dessus le chien paresseux. Si vous pouvez traduire ceci, vous connaissez bien votre alphabet."" German: ""Der schnelle braune Fuchs springt über den faulen Hund. Wenn Sie dies übersetzen können, kennen Sie Ihr Alphabet gut."""


In [10]:
# Define the prompts

prompt_1 = """Evaluate this Python function for correctness and efficiency:
            def fibonacci(n):
            if n <= 0:
            return 0
            elif n == 1:
            return 1
            else:
            return fibonacci(n-1) + fibonacci(n-2)"""
prompts = [prompt_1]
model_ids = [MIXTRAL,
            DEFAULT_MODEL,
            # MISTRAL_SMALL
                ]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    for model_id in model_ids:
        body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 514,
            "prompt": prompt,
            "stop": ["</s>"]
        })
        response = bedrock.invoke_model(
        body = body,
        modelId = model_id)
        response_body = json.loads(response.get("body").read())
        responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Mixtral 8x7b Analytical Prompt": [responses[0]],
    "Mistral Large Analytical Prompt": [responses[1]],
    # "Mistral Small Analytical Prompt": [responses[2]]
    

})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Mixtral 8x7b Analytical Prompt,Mistral Large Analytical Prompt
"Correctness: The function is not correct because it does not handle the case when n is 2, returning 0 instead of 1. Also, for large values of n, the function will exceed the maximum recursion depth due to multiple nested calls. Efficiency: The function's efficiency is very poor due to the repeated computation of Fibonacci numbers. For example, when calculating fibonacci(5), the function calls fibonacci(4) and fibonacci(3), which in turn call fibonacci(3) and fibonacci(2), and so on. This leads to an exponential time complexity. To improve the efficiency, we can use dynamic programming to store and reuse the calculated Fibonacci numbers. Here's an optimized version of the function: def fibonacci(n, memo={}): if n <= 0: return 0 elif n == 1: return 1 elif n in memo: return memo[n] else: result = fibonacci(n-1, memo) + fibonacci(n-2, memo) memo[n] = result return result This version has a time complexity of O(n) and a space complexity of O(n), making it much more efficient than the original implementation.","The function is correct in that it calculates the Fibonacci sequence. However, it's not efficient because it uses recursion to calculate each number in the sequence, which results in a lot of redundant calculations. For example, to calculate the 5th number in the sequence, it first calculates the 4th and 3rd numbers, but to calculate the 4th number, it already calculated the 3rd number. This redundancy leads to an exponential time complexity of O(2^n). A more efficient approach would be to use dynamic programming or iteration to calculate each number in the sequence only once."


## Conclusion
In this notebook, we discussed the common pitfalls that can arise when dealing with prompts and large language models (LLMs). These pitfalls include hallucination and factual inaccuracies, lack of coherence and logical reasoning in generated text, difficulty with complex multi-step tasks, and misunderstanding user intent.

We then explored how to create strong prompts and the importance of prompt engineering techniques to mitigate these issues. The key prompt engineering techniques covered were zero-shot prompting, few-shot prompting, and chain-of-thought (CoT) prompting.

Zero-shot prompting relies solely on the model's pre-existing knowledge without any additional examples. Few-shot prompting provides a small number of examples to guide the model's outputs. CoT prompting encourages the model to explain its reasoning process step-by-step, which is particularly useful for complex reasoning tasks.

Finally, we discussed the differences between the Mixtral 8x7b and Mistral Large models and their respective use cases. While Mixtral 8x7b can provide analytical responses, Mistral Large offers more thorough and contextually aware outputs, especially for multilingual tasks.
